# Справочники TVI API
Пример ноутбука для работы со справочниками TV Index API.

* [Инициализация](#Инициализация)
* [Доступные атрибуты для параметров](#Доступные-атрибуты-для-параметров)
* [Социально-демографические переменные](#Социально-демографические-переменные)
* [Телеканалы](#Телеканалы)
* [Место просмотра](#Место-просмотра)
* [Временные интервалы](#Временные-интервалы)
* [Даты](#Даты)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter 

In [3]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.mediavortex import catalogs as cwc
from mediascope_api.mediavortex import tasks as cwts


mnet = msnet.MediascopeApiNetwork()
cats = cwc.MediaVortexCats()

Запрошены записи: 0 - 257
Всего найдено записей: 257



## Доступные атрибуты для параметров

При формировании задания можно указать следующие параметры:

- **filters** - Фильтры
- **slices** - Срезы
- **statistics** - Статистики

Доступные атрибуты и допустимые для них значения можно получить методом `get_timeband_unit()`:

In [4]:
cats.get_timeband_unit()

{'statistics': ['Reach000',
  'ReachPer',
  'AvReach000',
  'AvReachPer',
  'Rtg000',
  'RtgPer',
  'TgAffinPer',
  'TAud',
  'ATV',
  'ATVReach',
  'Share',
  'Universe000',
  'Sample',
  'TTVRtg000',
  'TTVRtgPer',
  'TTVReach000',
  'TTVReachPer',
  'TTVAvReach000',
  'TTVAvReachPer',
  'TTVATV',
  'TTVATVReach'],
 'slices': ['researchDate',
  'researchWeek',
  'researchMonth',
  'researchQuarter',
  'researchYear',
  'researchHalfYear',
  'researchWeekDay',
  'researchDayType',
  'locationId',
  'locationName',
  'locationEName',
  'timeBand1',
  'timeBand5',
  'timeBand10',
  'timeBand15',
  'timeBand30',
  'timeBand60',
  'tvCompanyId',
  'tvCompanyName',
  'tvCompanyEName',
  'tvCompanyHoldingId',
  'tvCompanyHoldingName',
  'tvCompanyHoldingEName',
  'tvCompanyMediaHoldingId',
  'tvCompanyMediaHoldingName',
  'tvCompanyMediaHoldingEName',
  'tvNetId',
  'tvNetName',
  'tvNetEName',
  'sex',
  'age',
  'education',
  'work',
  'persNum',
  'spendingsOnFood',
  'tvNum',
  'ageGro

## Социально-демографические переменные

In [5]:
# Получим список всех демографических переменных
cats.get_tv_demo_attribute()

Запрошены записи: 0 - 257
Всего найдено записей: 257



,id,valueId,name,colName,valueName
0,31,1,Дача,Dacha,Да
1,31,0,Дача,Dacha,Нет
2,108,1,Наличие дачи с ТВ,WghDachaTvExist,есть дача и есть ТВ на даче
3,108,2,Наличие дачи с ТВ,WghDachaTvExist,нет дачи или нет ТВ на даче
4,47,1,Доход шкала (с 2014),IncomeScale201401,0 - 5000
...,...,...,...,...,...
252,2,10,Возраст,Age,10
253,2,89,Возраст,Age,89
254,2,8,Возраст,Age,8
255,18,0,Возраст детей: 2-3 года,KidsAge4,Нет


In [6]:
# Так как по умолчанию в jupyter большие таблицы выводятся не полнотстью, 
# справочник можно экспортировать в Excel и работать с ним там

# Экспорт справочника в Excel. Файл сохраняется в директорию, в которой находится этот ноутбук.
demo_cat = cats.get_tv_demo_attribute()
writer = pd.ExcelWriter('demo_cat.xlsx')
demo_cat.to_excel(writer, 'demo_cat', index=False)
writer.save()

Запрошены записи: 0 - 257
Всего найдено записей: 257



In [7]:
# Получим список некоторых демографических переменных по названию
cats.get_tv_demo_attribute(names=['образование', 'занятость', 'Население 100+'])

Запрошены записи: 0 - 9
Всего найдено записей: 9



,id,valueId,name,colName,valueName
0,3,1,Образование,Education,начальное
1,3,2,Образование,Education,среднее
2,3,3,Образование,Education,высшее
3,3,99,Образование,Education,затрудняюсь ответить (з/о)
4,4,1,Занятость,Work,Работает полный день
5,4,3,Занятость,Work,Не работает
6,4,2,Занятость,Work,Работает неполный день
7,118,1,Население 100+,Cube100Plus100Minus,100+
8,118,2,Население 100+,Cube100Plus100Minus,100-


Демографические переменные, которые используются в заданиях, должны быть прописаны строго в следующем формате: 

__demoVariable__

То есть первая буква всегда строчная. Пример: __wghSuburbAgeGroup__

## Телеканалы

In [8]:
# Выведем справочник сетей
cats.get_tv_net()

Запрошены записи: 0 - 0
Всего найдено записей: 0



""


In [9]:
# Выведем справочник телекомпаний
cats.get_tv_company()

Запрошены записи: 0 - 48
Всего найдено записей: 48



,tvChannelId,tvNetId,regionId,tvCompanyGroupId,tvCompanyCategoryId,tvCompanyMediaType,tvCompanyHoldingId,tvCompanyMediaHoldingId,tvThematicId,isChannelTotal,isSetTotal,isThematicTotal,id,name,ename,status,information,monitoringType
0,797,258,99,0,0,T,0,1000034,6,True,True,False,1875,ЗВЕЗДА (СЕТЕВОЕ ВЕЩАНИЕ),ZVEZDA (NETWORK BROADCASTING),A,,M
1,7612,407,99,0,0,T,0,0,0,False,True,False,12319,OTHER TV SET (СЕТЕВОЕ ВЕЩАНИЕ),OTHER TV SET (NETWORK BROADCASTING),A,,M
2,5768,10,99,0,0,T,0,1000021,10,True,True,False,6859,МИР (СЕТЕВОЕ ВЕЩАНИЕ),MIR (NETWORK BROADCASTING),A,,M
3,5880,326,99,0,0,T,8,1000017,11,True,True,False,975,МАТЧ ТВ (СЕТЕВОЕ ВЕЩАНИЕ),MATCH TV (NETWORK BROADCASTING),A,,M
4,781,2,99,0,0,T,6,0,6,True,True,False,1859,РОССИЯ 1 (СЕТЕВОЕ ВЕЩАНИЕ),ROSSIYA 1 (NETWORK BROADCASTING),A,,M
5,6052,335,99,0,0,T,19,1000003,8,True,True,False,1830,DISCOVERY CHANNEL (ДО 31/12/2019) (СЕТЕВОЕ ВЕЩ...,DISCOVERY CHANNEL (TILL 31/12/2019) (NETWORK B...,A,,M
6,784,4,99,0,0,T,8,0,6,True,True,False,1862,НТВ (СЕТЕВОЕ ВЕЩАНИЕ),NTV (NETWORK BROADCASTING),A,,M
7,4085,84,99,0,0,T,0,1000007,6,True,True,False,5297,СОЛНЦЕ (СЕТЕВОЕ ВЕЩАНИЕ),SOLNTSE (NETWORK BROADCASTING),A,,M
8,8887,502,99,0,0,T,6,0,8,True,True,False,13285,СОЛОВЬЁВLIVE (СЕТЕВОЕ ВЕЩАНИЕ),SOLOVIEVLIVE (NETWORK BROADCASTING),A,,M
9,787,255,99,0,0,T,19,1000040,8,True,True,False,1865,ЧЕ (СЕТЕВОЕ ВЕЩАНИЕ),CHE (NETWORK BROADCASTING),A,,M


In [10]:
# Экспорт справочника в Excel. Файл сохраняется в директорию, в которой находится этот ноутбук.
comp_cat = cats.get_tv_company()
writer = pd.ExcelWriter('tvcomp_cat.xlsx')
comp_cat.to_excel(writer, 'tvcomp_cat', index=False)
writer.save()

Запрошены записи: 0 - 48
Всего найдено записей: 48



In [11]:
# Поиск по справочнику телекомпаний 
cats.get_tv_company(name=["росс"])

Запрошены записи: 0 - 4
Всего найдено записей: 4



,tvChannelId,tvNetId,regionId,tvCompanyGroupId,tvCompanyCategoryId,tvCompanyMediaType,tvCompanyHoldingId,tvCompanyMediaHoldingId,tvThematicId,isChannelTotal,isSetTotal,isThematicTotal,id,name,ename,status,information,monitoringType
0,781,2,99,0,0,T,6,0,6,True,True,False,1859,РОССИЯ 1 (СЕТЕВОЕ ВЕЩАНИЕ),ROSSIYA 1 (NETWORK BROADCASTING),A,,M
1,786,256,99,1,1,T,6,0,6,True,True,False,1864,РОССИЯ 2 (СЕТЕВОЕ ВЕЩАНИЕ),ROSSIYA 2 (NETWORK BROADCASTING),A,,M
2,785,13,99,0,0,T,6,0,6,True,True,False,1863,РОССИЯ К (СЕТЕВОЕ ВЕЩАНИЕ),ROSSIYA K (NETWORK BROADCASTING),A,,M
3,782,260,99,0,0,T,6,0,2,True,True,False,1860,РОССИЯ 24 (СЕТЕВОЕ ВЕЩАНИЕ),ROSSIYA 24 (NETWORK BROADCASTING),A,,M


In [12]:
# Справочник холдингов
cats.get_tv_company_holding()

Запрошены записи: 0 - 27
Всего найдено записей: 27



,id,name,ename
0,24,ГПМ РАДИО,GPM RADIO
1,12,КОНЦЕРН РАДИО-ЦЕНТР,CONCERN RADIO-CENTRE
2,10,KRUTOY MEDIA,KRUTOY MEDIA
3,16,ЛИДЕР СВЯЗИ,LIDER SVYAZI
4,28,M10 MEDIA,M10 MEDIA
5,21,"ДОРОЖНОЕ РАДИО, РАДИО ДЛЯ ДВОИХ, РАДИО ВАНЯ","DOROZHNOE RADIO, RADIO DLYA DVOIKH, RADIO VANYA"
6,23,ИЗЮМ,IZYUM
7,4,ВЕЩАТЕЛЬНАЯ КОРПОРАЦИЯ ПРОФ-МЕДИА,BROADCASTING CORPORATION PROF MEDIA
8,19,НАЦИОНАЛЬНАЯ МЕДИА ГРУППА,NATSIONALNAYA MEDIA GRUPPA
9,6,ВГТРК,VGTRK


In [13]:
# Справочник медиахолдингов
cats.get_tv_company_media_holding()

Запрошены записи: 0 - 39
Всего найдено записей: 39



,id,name,ename
0,1000040,СТС-МЕДИА,STS-MEDIA
1,1000038,ZEE ENTERTAINMENT ENTERPRISRS Ltd.,ZEE ENTERTAINMENT ENTERPRISRS LTD.
2,1000039,ПЕРВЫЙ КАНАЛ. ВСЕМИРНАЯ СЕТЬ,PERVY KANAL. VSEMIRNAYA SET
3,1000024,НАЦИОНАЛЬНАЯ СПУТНИКОВАЯ КОМПАНИЯ,NATSIONALNAYA SPUTNIKOVAYA KOMPANIYA
4,1000025,НКС Медиа,NKS Media
5,1000033,ТОЧКА РОСЫ,TOCHKA ROSY
6,1000032,ТАЙМ МЕДИА ГРУПП,TAYM MEDIA GRUPP
7,1000017,ГАЗПРОМ-МЕДИА РАЗВЛЕКАТЕЛЬНОЕ ТЕЛЕВИДЕНИЕ,GAZPROM-MEDIA RAZVLEKATELNOE TELEVIDENIE
8,1000015,WARNER MEDIA,WARNER MEDIA
9,1000013,VIACOM,VIACOM


In [14]:
# Справочник жанров телекомпаний
cats.get_tv_company_thematic()

Запрошены записи: 0 - 17
Всего найдено записей: 17



,id,name,ename
0,10,Социально-политические,Социально-политические
1,14,Хобби и увлечения,Хобби и увлечения
2,2,Информационный,Информационный
3,16,Этнические,Этнические
4,4,Музыкальный,Музыкальный
5,3,Кино и сериалы,Кино и сериалы
6,9,Религия,Религия
7,5,Новости,Новости
8,13,Телемагазины,Телемагазины
9,12,Стиль жизни,Стиль жизни


## Место просмотра

In [15]:
# Справочник мест просмотра
cats.get_tv_location()

Запрошены записи: 0 - 2
Всего найдено записей: 2



,id,name,ename
0,2,Дача,Dacha
1,1,Дом,Home


## Временные интервалы

In [16]:
# получить справочник временных интервалов
cats.get_tv_time_band()

,value,name
0,timeBand30,timeBand30
1,timeBand60,timeBand60
2,timeBand1,timeBand1
3,timeBand5,timeBand5
4,timeBand10,timeBand10
5,timeBand15,timeBand15


## Даты

In [17]:
# дни недели
cats.get_tv_day_week()

Запрошены записи: 0 - 7
Всего найдено записей: 7



,id,name,ename
0,1,понедельник,monday
1,2,вторник,tuesday
2,3,среда,wednesday
3,4,четверг,thursday
4,5,пятница,friday
5,6,суббота,saturday
6,7,воскресение,sunday


In [18]:
# тип дня
cats.get_tv_research_day_type()

Запрошены записи: 0 - 4
Всего найдено записей: 4



,id,name,ename
0,W,рабочий день,working day
1,H,праздничный день,public holiday
2,E,выходной день,weekend
3,A,день траура,day of mourning


In [19]:
# получить календарь
cats.get_tv_calendar()

Запрошены записи: 0 - 10592
Всего найдено записей: 10592



,researchDate,researchDayTypeId,id
0,2022-02-15,W,2022-02-15
1,2022-02-20,E,2022-02-20
2,2022-09-09,W,2022-09-09
3,2023-02-19,E,2023-02-19
4,2023-10-28,E,2023-10-28
...,...,...,...
9587,2021-03-27,E,2021-03-27
9588,2014-12-31,W,2014-12-31
9589,2023-06-09,W,2023-06-09
9590,2023-06-27,W,2023-06-27
